In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git" -q
!pip install --no-deps trl peft accelerate bitsandbytes -q
!pip install --upgrade --force-reinstall pyarrow
!pip install datasets==4.3.0

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pyarrow-22.0.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (3.2 kB)
Using cached pyarrow-22.0.0-cp312-cp312-manylinux_2_28_x86_64.whl (47.7 MB)
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 22.0.0
    Uninstalling pyarrow-22.0.0:
      Successfully uninstalled pyarrow-22.0.0


In [ ]:
%cd /content/drive/MyDrive/NLP-LLM-FINETUNING/LORA/datasets

/content/drive/MyDrive/NLP-LLM-FINETUNING/LORA/datasets


In [ ]:
!git clone https://huggingface.co/datasets/Naholav/CodeGen-Deep-5K

fatal: destination path 'CodeGen-Deep-5K' already exists and is not an empty directory.


In [ ]:
!git clone https://huggingface.co/datasets/Naholav/CodeGen-Diverse-5K

fatal: destination path 'CodeGen-Diverse-5K' already exists and is not an empty directory.


In [ ]:
%cd /content/drive/MyDrive/NLP-LLM-FINETUNING/LORA/model

/content/drive/MyDrive/NLP-LLM-FINETUNING/LORA/model


In [ ]:
!git clone https://huggingface.co/Qwen/Qwen2.5-Coder-1.5B-Instruct

fatal: destination path 'Qwen2.5-Coder-1.5B-Instruct' already exists and is not an empty directory.


In [ ]:
%cd /content/drive/MyDrive/NLP-LLM-FINETUNING/LORA

/content/drive/MyDrive/NLP-LLM-FINETUNING/LORA


In [ ]:
import os
import json
import torch
from datetime import datetime
from unsloth import FastLanguageModel, is_bfloat16_supported
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments, EarlyStoppingCallback
from transformers.trainer_callback import TrainerCallback

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
TRAINING_MODE = "diverse"

USE_REASONING = True

DEEP_DATASET = "Naholav/CodeGen-Deep-5K"
DIVERSE_DATASET = "Naholav/CodeGen-Diverse-5K"

In [ ]:
if USE_REASONING:
  SYSTEM_PROMPT = "You are an expert Python programmer. Please read the problem carefully before writing any Python code."
else:
  SYSTEM_PROMPT = "You are an expert programmer. Use <think> tags for reasoning before writing code."

In [ ]:
LORA_R = 32
LORA_ALPHA = 64  # r * 2
LORA_DROPOUT = 0.05

In [ ]:
LEARNING_RATE = 2e-4
EPOCHS = 3
BATCH_SIZE = 16
GRADIENT_ACCUMULATION = 1
MAX_LENGTH = 1024 if not USE_REASONING else 8192

WEIGHT_DECAY = 0.01
WARMUP_RATIO = 0.04
SEED = 42

In [ ]:
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
DIRECTORY_PATH = "/content/drive/MyDrive/NLP-LLM-FINETUNING/LORA"
OUTPUT_DIR = f"{DIRECTORY_PATH}/training_models/{TRAINING_MODE}/{TRAINING_MODE}_{timestamp}"
LOG_DIR = f"{DIRECTORY_PATH}/logs/{TRAINING_MODE}/{TRAINING_MODE}_{timestamp}"
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)

In [ ]:
class LogCallback(TrainerCallback):
    """Loss'ları hem ekrana hem dosyaya yaz"""
    def __init__(self, log_file):
        self.log_file = log_file

    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs:
            with open(self.log_file, 'a') as f:
                log_entry = {
                    "step": state.global_step,
                    "epoch": round(state.epoch, 2) if state.epoch else 0,
                    **logs
                }
                f.write(json.dumps(log_entry) + "\n")

            if "loss" in logs:
                print(f"📊 [Step {state.global_step}] Train Loss: {logs['loss']:.4f}")
            if "eval_loss" in logs:
                print(f"📈 [Step {state.global_step}] Eval Loss: {logs['eval_loss']:.4f}")

In [ ]:
print("Model yükleniyor...")

max_seq_length = MAX_LENGTH
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Qwen2.5-Coder-1.5B-Instruct",
    max_seq_length=max_seq_length,
    dtype=None,
    load_in_4bit=False,
)

print("Model yüklendi!")
print(f"Max sequence length: {max_seq_length}")
print(f"Device: {model.device}")

Model yükleniyor...
==((====))==  Unsloth 2025.12.1: Fast Qwen2 patching. Transformers: 4.57.2.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Model yüklendi!
Max sequence length: 8192
Device: cuda:0


In [ ]:
FastLanguageModel.for_inference(model)
if not USE_REASONING:
  messages = [
      {"role": "system", "content": "You are an expert Python programmer. Please read the problem carefully before writing any Python code."},
      {"role": "user", "content": "Write a Python function to check if a number is prime."}
  ]
else:
  messages = [
      {"role": "system", "content": "You are an expert programmer. Use <think> tags for reasoning before writing code."},
      {"role": "user", "content": "Write a Python function to check if a number is prime."}
  ]

inputs = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to("cuda")
outputs = model.generate(input_ids=inputs, max_new_tokens=1024)
print("BAŞLANGIÇ MODELİ CEVABI:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


BAŞLANGIÇ MODELİ CEVABI:
system
You are an expert programmer. Use <think> tags for reasoning before writing code.
user
Write a Python function to check if a number is prime.
assistant
To check if a number is prime, we need to determine if it has any divisors other than 1 and itself. Here's a step-by-step approach to implement this in Python:

1. Define a function `is_prime` that takes an integer `num` as input.
2. Check if the number is less than 2. If so, return `False` because numbers less than 2 are not prime.
3. Iterate from 2 up to the square root of `num`. For each number `i`, check if `num` is divisible by `i`.
4. If `num` is divisible by any number `i` (other than 1 and itself), return `False`.
5. If no divisors are found, return `True`.

Here's the implementation of the `is_prime` function:

```python
import math

def is_prime(num):
    # Step 1: Check if num is less than 2
    if num < 2:
        return False
    
    # Step 2: Check for factors from 2 to the square root of n

In [ ]:
print("\nLoRA uygulanıyor...")
model = FastLanguageModel.get_peft_model(
    model,
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=SEED,
)

print("LoRA uygulandı!")

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.



LoRA uygulanıyor...


Unsloth 2025.12.1 patched 28 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


LoRA uygulandı!


In [ ]:
print("Dataset yükleniyor...")
dataset_name = DEEP_DATASET if TRAINING_MODE == "deep" else DIVERSE_DATASET
print(f"Dataset: {dataset_name}")

dataset = load_dataset(dataset_name)
dataset = dataset["train"].train_test_split(test_size=0.1, seed=42)
print(f"Train: {len(dataset['train'])} örnekler")
print(f"Test: {len(dataset['test'])} örnekler")

Dataset yükleniyor...
Dataset: Naholav/CodeGen-Diverse-5K
Train: 4500 örnekler
Test: 500 örnekler


In [ ]:
def formatting_func(examples):
    inputs = examples["input"]
    outputs = examples["output"] if USE_REASONING else examples["solution"]

    texts = []

    for input_text, output_text in zip(inputs, outputs):
        text = f"<|im_start|>system\n{SYSTEM_PROMPT}<|im_end|>\n"
        text += f"<|im_start|>user\n{input_text}<|im_end|>\n"
        text += f"<|im_start|>assistant\n{output_text}<|im_end|>"
        texts.append(text)

    return texts

In [ ]:
print("Training ayarları hazırlanıyor...")
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION,

    # Optimizer & Scheduler
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    warmup_ratio=WARMUP_RATIO,
    optim="adamw_8bit",
    lr_scheduler_type="cosine",

    # Logging & Saving
    logging_steps=20,
    eval_steps=100,
    save_steps=100,
    save_total_limit=3,

    # Evaluation & Model Selection
    eval_strategy="steps",
    save_strategy="steps",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,

    # Optimization
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    max_grad_norm=1.0,

    # Misc
    report_to="tensorboard",
    seed=SEED,
    dataloader_num_workers=2,
)

print(f"Effective batch size: {BATCH_SIZE * GRADIENT_ACCUMULATION}")
print(f"Precision: {'BF16' if is_bfloat16_supported() else 'FP16'}")

Training ayarları hazırlanıyor...
Effective batch size: 16
Precision: BF16


In [ ]:
print("\nTrainer hazırlanıyor...")
log_file = os.path.join(LOG_DIR, "training_log.jsonl")
log_callback = LogCallback(log_file)
early_stopping = EarlyStoppingCallback(early_stopping_patience=2)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    max_seq_length=max_seq_length,
    args=training_args,
    callbacks=[log_callback, early_stopping],
    packing=False,
    formatting_func=formatting_func
)

config = {
    "training_mode": TRAINING_MODE,
    "dataset": dataset_name,
    "use_reasoning": USE_REASONING,
    "base_model": "Qwen2.5-Coder-1.5B-Instruct",
    "framework": "unsloth",

    "lora_config": {
        "r": LORA_R,
        "alpha": LORA_ALPHA,
        "dropout": LORA_DROPOUT,
        "target_modules": ["q_proj", "k_proj", "v_proj", "o_proj",
                          "gate_proj", "up_proj", "down_proj"]
    },

    "training_config": {
        "learning_rate": LEARNING_RATE,
        "epochs": EPOCHS,
        "batch_size": BATCH_SIZE,
        "gradient_accumulation": GRADIENT_ACCUMULATION,
        "effective_batch_size": BATCH_SIZE * GRADIENT_ACCUMULATION,
        "max_length": MAX_LENGTH,
        "weight_decay": WEIGHT_DECAY,
        "warmup_ratio": WARMUP_RATIO,
        "optimizer": "adamw_8bit",
        "scheduler": "cosine",
        "early_stopping_patience": 2,
        "seed": SEED
    }
}

with open(os.path.join(OUTPUT_DIR, "config.json"), 'w') as f:
    json.dump(config, f, indent=2)

print("Trainer hazır!")


Trainer hazırlanıyor...
Trainer hazır!


In [ ]:
print("EĞİTİM BAŞLIYOR!")
gpu_stats = torch.cuda.get_device_properties(0)
print(f"GPU: {gpu_stats.name}")
print(f"Total Memory: {round(gpu_stats.total_memory / 1024 / 1024 / 1024, 2)} GB")
print(f"Compute Capability: {gpu_stats.major}.{gpu_stats.minor}\n")

trainer_stats = trainer.train()

print("EĞİTİM TAMAMLANDI!")


The model is already on multiple devices. Skipping the move to device specified in `args`.


EĞİTİM BAŞLIYOR!
GPU: NVIDIA A100-SXM4-80GB
Total Memory: 79.32 GB
Compute Capability: 8.0



==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 4,500 | Num Epochs = 3 | Total steps = 846
O^O/ \_/ \    Batch size per device = 16 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (16 x 1 x 1) = 16
 "-____-"     Trainable parameters = 36,929,536 of 1,580,643,840 (2.34% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
100,0.988000,0.978054
200,0.977300,0.953316
300,0.900300,0.947177
400,0.880800,0.939701
500,0.863700,0.930930
600,0.790600,0.945893
700,0.770900,0.945679


📊 [Step 20] Train Loss: 1.3358
📊 [Step 40] Train Loss: 1.0978
📊 [Step 60] Train Loss: 1.0624
📊 [Step 80] Train Loss: 0.9913
📊 [Step 100] Train Loss: 0.9880
📈 [Step 100] Eval Loss: 0.9781
📊 [Step 120] Train Loss: 1.0145
📊 [Step 140] Train Loss: 0.9996
📊 [Step 160] Train Loss: 0.9731
📊 [Step 180] Train Loss: 0.9698
📊 [Step 200] Train Loss: 0.9773
📈 [Step 200] Eval Loss: 0.9533
📊 [Step 220] Train Loss: 0.9869
📊 [Step 240] Train Loss: 0.9675
📊 [Step 260] Train Loss: 0.9610
📊 [Step 280] Train Loss: 0.9646
📊 [Step 300] Train Loss: 0.9003
📈 [Step 300] Eval Loss: 0.9472
📊 [Step 320] Train Loss: 0.8559
📊 [Step 340] Train Loss: 0.8600
📊 [Step 360] Train Loss: 0.8487
📊 [Step 380] Train Loss: 0.8802
📊 [Step 400] Train Loss: 0.8808
📈 [Step 400] Eval Loss: 0.9397
📊 [Step 420] Train Loss: 0.8976
📊 [Step 440] Train Loss: 0.8969
📊 [Step 460] Train Loss: 0.8426
📊 [Step 480] Train Loss: 0.8783
📊 [Step 500] Train Loss: 0.8637
📈 [Step 500] Eval Loss: 0.9309
📊 [Step 520] Train Loss: 0.8449
📊 [Step 540] Trai

In [ ]:
print("\nModel kaydediliyor...")
lora_dir = os.path.join(DIRECTORY_PATH, "lora_adapter")
model.save_pretrained(lora_dir)
tokenizer.save_pretrained(lora_dir)
print(f"LoRA adapter: {lora_dir}")


Model kaydediliyor...
LoRA adapter: /content/drive/MyDrive/NLP-LLM-FINETUNING/LORA/lora_adapter


In [ ]:
print("\nMerged model oluşturuluyor...")
merged_dir = os.path.join(OUTPUT_DIR, "merged_model")
model.save_pretrained_merged(merged_dir, tokenizer, save_method="merged_16bit")
print(f"Merged model (16bit): {merged_dir}")


Merged model oluşturuluyor...
Found HuggingFace hub cache directory: /root/.cache/huggingface/hub
Checking cache directory for required files...


Unsloth: Copying 1 files from cache to `/content/drive/MyDrive/NLP-LLM-FINETUNING/LORA/training_models/diverse/diverse_20251208_171913/merged_model`: 100%|██████████| 1/1 [00:11<00:00, 11.20s/it]


Successfully copied all 1 files from cache to `/content/drive/MyDrive/NLP-LLM-FINETUNING/LORA/training_models/diverse/diverse_20251208_171913/merged_model`
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Merging weights into 16bit: 100%|██████████| 1/1 [00:18<00:00, 18.99s/it]


Unsloth: Merge process complete. Saved to `/content/drive/MyDrive/NLP-LLM-FINETUNING/LORA/training_models/diverse/diverse_20251208_171913/merged_model`
Merged model (16bit): /content/drive/MyDrive/NLP-LLM-FINETUNING/LORA/training_models/diverse/diverse_20251208_171913/merged_model


In [ ]:
print("\nHızlı inference testi yapılıyor...\n")

FastLanguageModel.for_inference(model)  # Inference mode

test_prompt = """Write a Python function that takes a list of integers and returns the sum of all even numbers."""

messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": test_prompt}
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
).to("cuda")

outputs = model.generate(
    input_ids=inputs,
    max_new_tokens=1024,
    temperature=0.6,
    do_sample=True,
    top_p=0.9,
    pad_token_id=tokenizer.pad_token_id
)

response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("TEST ÇIKTISI:")
print(response)

print("\nGPU Memory Kullanımı:")
print(f"Allocated: {torch.cuda.memory_allocated(0)/1024**3:.2f} GB")
print(f"Reserved: {torch.cuda.memory_reserved(0)/1024**3:.2f} GB")

print("\n\nTüm işlemler tamamlandı! Başarılar!")


Hızlı inference testi yapılıyor...

TEST ÇIKTISI:
system
You are an expert Python programmer. Please read the problem carefully before writing any Python code.
user
Write a Python function that takes a list of integers and returns the sum of all even numbers.
assistant
<think>
This is a straightforward problem where I need to find all even numbers in a given list and compute their sum. The key insight is that for each number, I can check if it's divisible by 2 (even) - this is a direct mathematical property. 

I should consider edge cases first: what if the list is empty? Then the sum would be 0 since there are no numbers to add. What about negative even numbers? The problem doesn't specify whether we include negative numbers or not, but typically when dealing with sums of numbers, we usually assume positive values unless explicitly stated otherwise.

Let me test this approach with concrete examples. For [1, 2, 3, 4], the even numbers are 2 and 4, so the sum would be 6. For [-2, 2, -4

In [ ]:
import random

num_samples = 3
test_indices = random.sample(range(len(dataset["test"])), num_samples)
test_samples = dataset["test"].select(test_indices)

print("Test Analizi")

FastLanguageModel.for_inference(model)

for i, sample in enumerate(test_samples):
  input_text = sample["input"]
  ground_truth = sample["solution"]

  messages = [
      {"role": "system", "content": SYSTEM_PROMPT},
      {"role": "user", "content": input_text}
  ]

  inputs = tokenizer.apply_chat_template(
      messages,
      tokenize=True,
      add_generation_prompt=True,
      return_tensors="pt"
  ).to("cuda")

  outputs = model.generate(
        input_ids=inputs,
        max_new_tokens=1024,
        temperature=0.6,
        top_p=0.9,
        do_sample=True
  )

  prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)

  print(f"\nÖRNEK {i+1}:")
  print(f"SORU:\n{input_text[:200]}...")
  print(f"|nMODEL TAHMİNİ:\n{prediction.split('assistant')[-1].strip()}")
  print(f"GERÇEK CEVAP (Referans):\n{ground_truth[:200]}...")

Test Analizi

ÖRNEK 1:
SORU:
Create a function `longer` that accepts a string and sorts the words in it based on their respective lengths in an ascending order. If there are two words of the same lengths, sort them alphabetically...
|nMODEL TAHMİNİ:
<think>
This problem requires sorting words by length first (ascending), then alphabetically if lengths are equal. The key insight is that we need to split the input into individual words, process each word individually, then reconstruct the sorted result.

Let me break this down into subgoals:
1. Split the input string into individual words
2. For each word, determine its length and compare with others
3. Sort primarily by length (ascending) but secondarily by alphabetical order
4. Reconstruct the final string

I should consider different approaches. One straightforward approach would be to iterate through all words once, keeping track of both length and original index. Then I could sort using a custom comparator that compares lengths firs